# Tutoriel interactif sur l'utilisation de l'API Web pour les opérations logiques

Ce notebook présente un tutoriel interactif sur l'utilisation de l'API Web pour effectuer des opérations logiques. Vous apprendrez à:

1. Vous connecter à l'API Web
2. Convertir des textes en ensembles de croyances formalisés
3. Exécuter des requêtes logiques sur ces ensembles de croyances
4. Générer des requêtes pertinentes
5. Interpréter les résultats des analyses

## Prérequis

Pour exécuter ce notebook, vous devez avoir installé:
- Python 3.8 ou supérieur
- Requests
- Pandas (pour l'affichage des résultats)
- Matplotlib (pour la visualisation)

Vous devez également disposer d'une clé API pour accéder à l'API Web des agents logiques.

## 1. Configuration de l'environnement

Commençons par importer les bibliothèques nécessaires et configurer l'environnement.

In [ ]:
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
from typing import Dict, List, Any, Optional, Union
from IPython.display import display, HTML

# Configuration de l'affichage
pd.set_option('display.max_colwidth', None)
plt.style.use('ggplot')

## 2. Client API pour les agents logiques

Nous allons créer une classe client pour interagir avec l'API Web des agents logiques.

In [ ]:
class LogicAgentsAPI:
    """
    Client pour l'API Web des agents logiques.
    """
    
    def __init__(self, base_url: str, api_key: str):
        """
        Initialise le client API.
        
        Args:
            base_url: URL de base de l'API
            api_key: Clé API pour l'authentification
        """
        self.base_url = base_url.rstrip('/')
        self.api_key = api_key
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }
        print(f"Client API initialisé avec l'URL de base: {base_url}")
    
    def create_belief_set(self, text: str, logic_type: str, options: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        """
        Convertit un texte en ensemble de croyances.
        
        Args:
            text: Texte à convertir
            logic_type: Type de logique ("propositional", "first_order", "modal")
            options: Options de conversion (optionnel)
            
        Returns:
            Réponse de l'API contenant l'ensemble de croyances créé
        """
        if options is None:
            options = {"include_explanation": True}
        
        payload = {
            "text": text,
            "logic_type": logic_type,
            "options": options
        }
        
        print(f"Conversion du texte en ensemble de croyances ({logic_type})...")
        response = self._make_request("POST", "/api/logic/belief-set", payload)
        
        belief_set_id = response["belief_set"]["id"]
        print(f"Ensemble de croyances créé avec ID: {belief_set_id}")
        
        return response
    
    def execute_query(self, belief_set_id: str, query: str, logic_type: str, options: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        """
        Exécute une requête sur un ensemble de croyances.
        
        Args:
            belief_set_id: ID de l'ensemble de croyances
            query: Requête à exécuter
            logic_type: Type de logique
            options: Options d'exécution (optionnel)
            
        Returns:
            Réponse de l'API contenant le résultat de la requête
        """
        if options is None:
            options = {"include_explanation": True}
        
        payload = {
            "belief_set_id": belief_set_id,
            "query": query,
            "logic_type": logic_type,
            "options": options
        }
        
        print(f"Exécution de la requête: {query}")
        response = self._make_request("POST", "/api/logic/query", payload)
        
        result = response["result"]["result"]
        print(f"Résultat: {result}")
        
        return response
    
    def generate_queries(self, belief_set_id: str, text: str, logic_type: str, options: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        """
        Génère des requêtes pertinentes pour un ensemble de croyances.
        
        Args:
            belief_set_id: ID de l'ensemble de croyances
            text: Texte source
            logic_type: Type de logique
            options: Options de génération (optionnel)
            
        Returns:
            Réponse de l'API contenant les requêtes générées
        """
        if options is None:
            options = {"max_queries": 5}
        
        payload = {
            "belief_set_id": belief_set_id,
            "text": text,
            "logic_type": logic_type,
            "options": options
        }
        
        print("Génération de requêtes pertinentes...")
        response = self._make_request("POST", "/api/logic/generate-queries", payload)
        
        queries = response["queries"]
        print(f"Requêtes générées: {queries}")
        
        return response
    
    def _make_request(self, method: str, endpoint: str, payload: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        """
        Effectue une requête à l'API.
        
        Args:
            method: Méthode HTTP (GET, POST, etc.)
            endpoint: Endpoint de l'API
            payload: Données à envoyer (optionnel)
            
        Returns:
            Réponse de l'API
            
        Raises:
            Exception: Si la requête échoue
        """
        url = f"{self.base_url}{endpoint}"
        
        try:
            if method == "GET":
                response = requests.get(url, headers=self.headers)
            elif method == "POST":
                response = requests.post(url, headers=self.headers, json=payload)
            else:
                raise ValueError(f"Méthode HTTP non supportée: {method}")
            
            response.raise_for_status()
            return response.json()
        
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la requête à l'API: {str(e)}")
            
            if hasattr(e, 'response') and e.response is not None:
                try:
                    error_data = e.response.json()
                    print(f"Détails de l'erreur: {json.dumps(error_data, indent=2)}")
                except:
                    print(f"Réponse d'erreur: {e.response.text}")
            
            raise

## 3. Initialisation du client API

Initialisons le client API avec l'URL de base et la clé API.

In [ ]:
# Configuration de l'API
API_BASE_URL = "https://api.example.com"  # Remplacez par l'URL réelle de l'API
API_KEY = "votre_clé_api"  # Remplacez par votre clé API

# Créer le client API
api_client = LogicAgentsAPI(API_BASE_URL, API_KEY)

## 4. Exemples d'utilisation

### 4.1 Logique propositionnelle

In [ ]:
# Texte à analyser
text_prop = """
Si le ciel est nuageux, alors il va pleuvoir.
Le ciel est nuageux.
"""

# Convertir le texte en ensemble de croyances
belief_set_response = api_client.create_belief_set(text_prop, "propositional")
belief_set_id = belief_set_response["belief_set"]["id"]

# Exécuter une requête
query = "pluie"
api_client.execute_query(belief_set_id, query, "propositional")

### 4.2 Logique du premier ordre

In [ ]:
# Texte à analyser
text_fol = """
Tous les hommes sont mortels.
Socrate est un homme.
"""

# Convertir le texte en ensemble de croyances
fol_belief_set_response = api_client.create_belief_set(text_fol, "first_order")
fol_belief_set_id = fol_belief_set_response["belief_set"]["id"]

# Exécuter une requête
fol_query = "Mortel(socrate)"
api_client.execute_query(fol_belief_set_id, fol_query, "first_order")

### 4.3 Logique modale

In [ ]:
# Texte à analyser
text_modal = """
Si une proposition est nécessairement vraie, alors elle est vraie.
Si une proposition est vraie, alors elle est possiblement vraie.
La somme des angles d'un triangle est nécessairement égale à 180 degrés.
"""

# Convertir le texte en ensemble de croyances
modal_belief_set_response = api_client.create_belief_set(text_modal, "modal")
modal_belief_set_id = modal_belief_set_response["belief_set"]["id"]

# Exécuter une requête
modal_query = "<>(somme_angles_triangle_180)"
api_client.execute_query(modal_belief_set_id, modal_query, "modal")

## 5. Exercices pratiques

### Exercice 1: Logique propositionnelle

Analysez le texte suivant avec l'API pour la logique propositionnelle:

```
Si l'économie se porte bien, alors le chômage diminue.
Si le chômage diminue, alors la satisfaction des citoyens augmente.
L'économie ne se porte pas bien.
```

Questions:
1. Quel est l'ensemble de croyances généré?
2. Peut-on conclure que la satisfaction des citoyens n'augmente pas?
3. L'ensemble de croyances est-il cohérent?

In [ ]:
# Votre code ici

### Exercice 2: Logique du premier ordre

Analysez le texte suivant avec l'API pour la logique du premier ordre:

```
Tous les oiseaux peuvent voler.
Les pingouins sont des oiseaux.
Tux est un pingouin.
```

Questions:
1. Quel est l'ensemble de croyances généré?
2. Peut-on conclure que Tux peut voler?
3. Comment pourriez-vous modifier l'ensemble de croyances pour refléter le fait que les pingouins sont une exception à la règle que tous les oiseaux peuvent voler?

In [ ]:
# Votre code ici

## 6. Conclusion

Dans ce tutoriel, nous avons exploré l'utilisation de l'API Web pour effectuer des opérations logiques. Nous avons appris à:

1. Convertir des textes en ensembles de croyances formalisés pour différents types de logique
2. Exécuter des requêtes logiques sur ces ensembles de croyances
3. Générer des requêtes pertinentes automatiquement

L'API Web offre une interface puissante pour intégrer des capacités de raisonnement logique dans vos applications, sans avoir à implémenter vous-même les algorithmes complexes de raisonnement logique.